In [24]:
# Standard imports
# This script creates an up to date spreadsheet with all the price info we need, which we can easily then paste into
# pre-formatted spreadsheet namned !!!name here!!! to return results
# You'll need to pip install pandas and numpy for this to work.
# I recommend you download Anaconda (an "environment" for python) which comes with everything we need already
# https://packaging.python.org/tutorials/installing-packages/
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

#This is a Jupyter notebook in Python. All that it requires is that you place this script
#in a folder with the other spreadsheets this came with, and execute (shift+enter) each cell in order
#please contact me on jabber for any questions

In [25]:
itemidDF = pd.read_excel('eveItemIDs.xlsx')
pricesDF = pd.read_json('https://esi.evetech.net/latest/markets/prices/?datasource=tranquility')
#holy fuck it's actually this easy
#this accomplished in two lines what that entire other abandoned script was supposed to do 


In [26]:
DFfinal = pd.merge(itemidDF,pricesDF,left_on='typeID',right_on='type_id')
#creates a DF with both prices and names using shared item IDs
#if new goods are added to the game, then the excel file "eveItemIDs.xlsx might have to be updated
#pricesDF goes straight to ESI so that will update automatically but just running this script

In [27]:
DFfinal = DFfinal.drop('Unnamed: 2',axis=1)
#cleans up our dataframe

In [28]:
pi_goods = pd.read_excel('PIgoods.xlsx')
goods0r = pi_goods['Level 0']
goods1r = pi_goods['Level 1']
goods2r = pi_goods['Level 2']
goods3r = pi_goods['Level 3']
goods4r = pi_goods['Level 4']
#this breaks up the lists so we can put them back together later
#this is not the best way to do this but it works I guess

In [29]:
goods0 = goods0r.dropna()
goods1 = goods1r.dropna()
goods2 = goods2r.dropna()
goods3 = goods3r.dropna()
goods4 = goods4r.dropna()
frames = [goods0,goods1,goods2,goods3,goods4]
goods = pd.concat(frames)
#this puts together the lists we need of all PI goods in a new object we can easily access

In [30]:
goodsDF = pd.DataFrame(goods,columns=['ItemName'])
#turns that object into a DF with an index namned so we can zip it

In [31]:
PIgoodsDF = pd.merge(goodsDF,DFfinal,how='left',left_on='ItemName',right_on='typeName')
#we zip the previous all items and all prices DF to the PI goods and attach the current price for each PI good

In [32]:
PIgoodsDF = PIgoodsDF.sort_values('ItemName')
#order by item name

In [33]:
adjustedPriceDF = PIgoodsDF.filter(['ItemName','adjusted_price'],axis=1)
averagePriceDF = PIgoodsDF.filter(['ItemName','average_price'],axis=1)
#CCP provides two prices, one a simple average of all market prices, and an "adjused" with limited documentation
#that's apparently supposed to be more immune to market manipulation

In [34]:
writer = pd.ExcelWriter('output.xlsx')
#just don't touch this

In [35]:
adjustedPriceDF.to_excel(writer,'adjustedPrices')
averagePriceDF.to_excel(writer,'averagePrices')